In [1]:
import os
import sys
import copy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
sys.path.append(os.path.join(os.path.pardir, 'gotmtool'))
from gotmtool import *

## Case parameters

In [2]:
casename = 'lsc_ymc22_sbl_bbl_v2_rf'
g = 9.81
cd = 1.25e-3
rhoa = 1.225
rhoo = 1026
Omega = 2.*np.pi/86400

lat = 45.
H = 30.
u10 = 8
U0 = 0.25
if 'rf' in casename:
    U0 *= -1.

amplitude = 1. # m
wavelength = 60. # m
tau = rhoa * cd * u10 * u10

f = 2*Omega*np.sin(np.deg2rad(lat))
dpdy = -f*U0
dhdy = float(dpdy/g)

### Stokes drift

Stokes drift in shallow water of depth $H$ is
$$
u_s = U_s\frac{\cosh[2k(z+H)]}{2\sinh^2(kH)}
$$
where $k$ is the wavenumber, $U_s=\sigma_w k a_w^2$ is the surface value of Stokes drift, with $\sigma_w$ the frequency and $a_w$ the amplitude.

The vertical shear of this Stokes drift is
$$
\frac{d u_s}{dz} = U_s k \frac{\sinh[2k(z+H)]}{\sinh^2(kH)}
$$

In [3]:
def get_stokes_drift(nlev, depth, wavelength, amplitude):
    wavenumber = 2. * np.pi / wavelength # 1/m
    frequency = np.sqrt(g * wavenumber * np.tanh(wavenumber * depth)) # 1/s
    us0 = amplitude**2 * wavenumber * frequency # m/s

    zi = -np.linspace(0,depth,nlev+1)
    z  = 0.5 * (zi[:-1] + zi[1:])
    us = us0 * np.cosh(2. * wavenumber * (z + depth)) / (2. * np.sinh(wavenumber * depth)**2)
    us_arr = us.reshape((1, us.size))
    vs_arr = np.zeros_like(us_arr)

    dusdz = us0 * wavenumber * np.sinh(2. * wavenumber * (zi + depth)) / np.sinh(wavenumber * depth)**2
    dusdz_arr = dusdz.reshape((1, dusdz.size))
    dvsdz_arr = np.zeros_like(dusdz_arr)
    
    return us_arr, vs_arr, dusdz_arr, dvsdz_arr, z, zi

## Create a model

In [4]:
m = Model(name='{:s}'.format(casename), environ=os.path.join(os.path.pardir, 'gotmtool', '.gotm_env.yaml'))

Take a look at what are defined in the environment file.

In [5]:
for key in m.environ:
    print('{:>15s}: {}'.format(key, m.environ[key]) )

   gotmdir_code: /data1/scratch/qingli/project/A2024_CmpFlux/gotm/code
   gotmdir_data: /data1/scratch/qingli/project/A2024_CmpFlux/gotm/data
  gotmdir_build: /data1/scratch/qingli/project/A2024_CmpFlux/gotm/build
    gotmdir_exe: /data1/scratch/qingli/project/A2024_CmpFlux/gotm/exe
    gotmdir_run: /data1/scratch/qingli/project/A2024_CmpFlux/gotm/run
 gotmdir_figure: /data1/scratch/qingli/project/A2024_CmpFlux/gotm/figure
   gotmdir_tool: /data1/scratch/qingli/project/A2024_CmpFlux/gotmtool


## Build the model

In [6]:
%%time
m.build()

CPU times: user 9.07 ms, sys: 45.7 ms, total: 54.8 ms
Wall time: 127 ms


## Configuration
Initialize the GOTM configuration

In [7]:
cfg = m.init_config()

Generating default configuration at '/data1/scratch/qingli/project/A2024_CmpFlux/gotm/run/lsc_ymc22_sbl_bbl_v2_rf/gotm.yaml'...
Done!


 ------------------------------------------------------------------------
 GOTM started on 2024/11/21 at 11:09:11
 ------------------------------------------------------------------------
    initialize_gotm
 ------------------------------------------------------------------------
        Reading configuration from: gotm.yaml
        configuring modules ....
    init_airsea_yaml
        done
    init_observations_yaml
    init_stokes_drift_yaml
        done
    init_turbulence_yaml
        done.
    init_cvmix_yaml
        done.
    init_meanflow_yaml
        done
    init_eqstate_yaml
    init_density()
            rho0=     1027.0000000000000     
        done.
 Your configuration has been written to /data1/scratch/qingli/project/A2024_CmpFlux/gotm/run/lsc_ymc22_sbl_bbl_v2_rf/gotm.yaml.
STOP 0


Update the configuration

In [8]:
# setup
title = 'Merging boundary layers'
cfg['title'] = title
cfg['location']['name'] = 'Idealized'
cfg['location']['latitude'] = lat
cfg['location']['longitude'] = 0.0
cfg['location']['depth'] = H
cfg['time']['start'] = '2000-01-01 00:00:00'
cfg['time']['stop']  = '2000-01-13 00:00:00'
cfg['time']['dt']    = 60.0


# output
cfg['output'] = {}
cfg['output']['gotm_out'] = {}
cfg['output']['gotm_out']['use'] = True
cfg['output']['gotm_out']['title'] = title
cfg['output']['gotm_out']['time_unit'] = 'dt'
cfg['output']['gotm_out']['time_step'] = 3
cfg['output']['gotm_out']['variables'] = [{}]
cfg['output']['gotm_out']['variables'][0]['source'] = '*'

# forcing
datadir = os.path.join(m.environ['gotmdir_data'], casename)
cfg['temperature']['method'] = 'file'
cfg['temperature']['file'] = os.path.join(datadir, 't_prof.dat')
cfg['temperature']['type'] = 'conservative'
cfg['salinity']['method'] = 'constant'
cfg['salinity']['constant_value'] = 35.0
cfg['surface']['fluxes']['heat']['method'] = 'constant'
cfg['surface']['fluxes']['heat']['constant_value'] = 0.0
cfg['surface']['fluxes']['tx']['method'] = 'constant'
cfg['surface']['fluxes']['tx']['constant_value'] = tau
cfg['surface']['fluxes']['ty']['method'] = 'constant'
cfg['surface']['fluxes']['ty']['constant_value'] = 0.0
cfg['surface']['swr']['method'] = 'constant'
cfg['surface']['precip']['method'] = 'constant'
cfg['mimic_3d']['ext_pressure']['type'] = 'elevation'
cfg['mimic_3d']['ext_pressure']['dpdy']['method'] = 'constant'
cfg['mimic_3d']['ext_pressure']['dpdy']['constant_value'] = dhdy
cfg['bottom']['h0b'] = 0.01
cfg['velocities']['u']['method'] = 'file'
cfg['velocities']['u']['column'] = 1
cfg['velocities']['v']['method'] = 'file'
cfg['velocities']['v']['column'] = 2

# Stokes drift
cfg['waves']['stokes_drift']['us']['method'] = 'file'
cfg['waves']['stokes_drift']['vs']['method'] = 'file'
cfg['waves']['stokes_drift']['us']['file'] = 'us_prof.dat'
cfg['waves']['stokes_drift']['us']['column'] = 1
cfg['waves']['stokes_drift']['vs']['file'] = 'us_prof.dat'
cfg['waves']['stokes_drift']['vs']['column'] = 2
cfg['waves']['stokes_drift']['dusdz']['method'] = 'file'
cfg['waves']['stokes_drift']['dvsdz']['method'] = 'file'
cfg['waves']['stokes_drift']['dusdz']['file'] = 'dusdz_prof.dat'
cfg['waves']['stokes_drift']['dusdz']['column'] = 1
cfg['waves']['stokes_drift']['dvsdz']['file'] = 'dusdz_prof.dat'
cfg['waves']['stokes_drift']['dvsdz']['column'] = 2

# EOS -- use linear
cfg['equation_of_state']['method'] = 'linear_custom'
cfg['equation_of_state']['rho0'] = rhoo
cfg['equation_of_state']['linear']['T0'] = 20.0
cfg['equation_of_state']['linear']['S0'] = 35.0
cfg['equation_of_state']['linear']['alpha'] = 2.0e-4
cfg['equation_of_state']['linear']['beta'] = 8.0e-4
cfg['equation_of_state']['linear']['cp'] = 3991.0

# water type (Jerlov IB)
cfg['light_extinction']['method'] = 'jerlov-ib'

### Turbulence closure scheme


In [9]:
# collect the configurations and the labels of the two runs
cfgs = []
labels = []

In [10]:
turbmethods = ['KPPLT-LF17', 'KPPLT-LF17-E', 'SMCLT-H15', 'SMCLT-KC04']
nlevels = [18, 36, 72, 144, 288]
trelax = [0., 60., 600., 3600.]

runs = []
for turbmethod in turbmethods:
    for nlevel in nlevels:
        for trlx in trelax:
            run = '{:s}_L{:g}_Rlx{:g}'.format(turbmethod, nlevel, trlx)
            runs.append(run)
print(runs)

['KPPLT-LF17_L18_Rlx0', 'KPPLT-LF17_L18_Rlx60', 'KPPLT-LF17_L18_Rlx600', 'KPPLT-LF17_L18_Rlx3600', 'KPPLT-LF17_L36_Rlx0', 'KPPLT-LF17_L36_Rlx60', 'KPPLT-LF17_L36_Rlx600', 'KPPLT-LF17_L36_Rlx3600', 'KPPLT-LF17_L72_Rlx0', 'KPPLT-LF17_L72_Rlx60', 'KPPLT-LF17_L72_Rlx600', 'KPPLT-LF17_L72_Rlx3600', 'KPPLT-LF17_L144_Rlx0', 'KPPLT-LF17_L144_Rlx60', 'KPPLT-LF17_L144_Rlx600', 'KPPLT-LF17_L144_Rlx3600', 'KPPLT-LF17_L288_Rlx0', 'KPPLT-LF17_L288_Rlx60', 'KPPLT-LF17_L288_Rlx600', 'KPPLT-LF17_L288_Rlx3600', 'KPPLT-LF17-E_L18_Rlx0', 'KPPLT-LF17-E_L18_Rlx60', 'KPPLT-LF17-E_L18_Rlx600', 'KPPLT-LF17-E_L18_Rlx3600', 'KPPLT-LF17-E_L36_Rlx0', 'KPPLT-LF17-E_L36_Rlx60', 'KPPLT-LF17-E_L36_Rlx600', 'KPPLT-LF17-E_L36_Rlx3600', 'KPPLT-LF17-E_L72_Rlx0', 'KPPLT-LF17-E_L72_Rlx60', 'KPPLT-LF17-E_L72_Rlx600', 'KPPLT-LF17-E_L72_Rlx3600', 'KPPLT-LF17-E_L144_Rlx0', 'KPPLT-LF17-E_L144_Rlx60', 'KPPLT-LF17-E_L144_Rlx600', 'KPPLT-LF17-E_L144_Rlx3600', 'KPPLT-LF17-E_L288_Rlx0', 'KPPLT-LF17-E_L288_Rlx60', 'KPPLT-LF17-E_L288_R

In [11]:
stokesdrift = {}
for nlevel in nlevels:
    stokesdrift['L{:g}'.format(nlevel)] = get_stokes_drift(nlevel, H, wavelength, amplitude)

In [12]:
def set_gotm_turbmethod(cfg, turbmethod):
    if turbmethod == 'KPPLT-LF17':
        cfg['turbulence']['turb_method'] = 'cvmix'
        cfg['cvmix']['surface_layer']['use'] = True
        cfg['cvmix']['surface_layer']['langmuir_method'] = 'lf17'
        cfg['cvmix']['bottom_layer']['use'] = True
        cfg['cvmix']['interior']['use'] = True
        cfg['cvmix']['interior']['background']['use'] = True
        cfg['cvmix']['interior']['shear']['use'] = True
    elif turbmethod == 'KPPLT-LF17-E':
        cfg['turbulence']['turb_method'] = 'cvmix'
        cfg['cvmix']['surface_layer']['use'] = True
        cfg['cvmix']['surface_layer']['langmuir_method'] = 'lf17'
        cfg['cvmix']['bottom_layer']['use'] = True
        cfg['cvmix']['interior']['use'] = True
        cfg['cvmix']['interior']['background']['use'] = True
        cfg['cvmix']['interior']['shear']['use'] = True
    elif turbmethod == 'SMCLT-H15':
        cfg['turbulence']['turb_method'] = 'second_order'
        cfg['turbulence']['tke_method'] = 'mellor_yamada'
        cfg['turbulence']['len_scale_method'] = 'mellor_yamada'
        cfg['turbulence']['scnd']['method'] =  'quasi_eq_h15'
        cfg['turbulence']['scnd']['scnd_coeff'] =  'kantha_clayson'
        cfg['turbulence']['turb_param']['length_lim'] = 'false'
        cfg['turbulence']['turb_param']['compute_c3'] = 'false'
        cfg['turbulence']['my']['e3'] = 5.0
        cfg['turbulence']['my']['e6'] = 6.0
    elif turbmethod == 'SMCLT-KC04':
        cfg['turbulence']['turb_method'] = 'second_order'
        cfg['turbulence']['tke_method'] = 'mellor_yamada'
        cfg['turbulence']['len_scale_method'] = 'mellor_yamada'
        cfg['turbulence']['scnd']['method'] =  'quasi_eq'
        cfg['turbulence']['scnd']['scnd_coeff'] =  'kantha_clayson'
        cfg['turbulence']['turb_param']['length_lim'] = 'false'
        cfg['turbulence']['turb_param']['compute_c3'] = 'false'
        cfg['turbulence']['my']['e3'] = 5.0
        cfg['turbulence']['my']['e6'] = 6.0
    else:
        raise ValueError('Unknown turbulence closure method: {:s}'.format(turbmethod))
    if turbmethod == 'KPPLT-LF17':
        cfg['waves']['stokes_drift']['coriolis_stokes_force'] = False
        cfg['velocities']['u']['file'] = os.path.join(datadir, 'u_prof_l.dat')
        cfg['velocities']['v']['file'] = os.path.join(datadir, 'u_prof_l.dat')
    else:
        cfg['waves']['stokes_drift']['coriolis_stokes_force'] = True
        cfg['velocities']['u']['file'] = os.path.join(datadir, 'u_prof_e.dat')
        cfg['velocities']['v']['file'] = os.path.join(datadir, 'u_prof_e.dat')   
    return cfg

def set_gotm_relax(cfg, trlx):
    if trlx == 0.0:
        cfg['temperature']['relax']['tau'] = 1.0e15
        cfg['velocities']['relax']['tau'] = 1.0e15
    else:
        cfg['temperature']['relax']['tau'] = trlx
        cfg['velocities']['relax']['tau'] = trlx
    return cfg

In [13]:
timeSD =  pd.date_range(cfg['time']['start'], freq='D', periods=1)
timeSD

DatetimeIndex(['2000-01-01'], dtype='datetime64[ns]', freq='D')

In [14]:
for run in runs:
    print(run)
    turbmethod, levstr, rlxstr = run.split('_')
    nlev = int(levstr[1:])
    rlx = float(rlxstr[3:])
    cfg['grid']['nlev']  = nlev
    cfg['output']['gotm_out']['k1_stop'] = nlev+1
    cfg['output']['gotm_out']['k_stop'] = nlev
    cfg = set_gotm_turbmethod(cfg, turbmethod)
    cfg = set_gotm_relax(cfg, rlx)
    cfgs.append(copy.deepcopy(cfg))
    labels.append(run)
    # dump Stokes drift
    us_arr, vs_arr, dusdz_arr, dvsdz_arr, z, zi = stokesdrift[levstr]
    run_dir = os.path.join(m.environ['gotmdir_run'], m.name, run)
    # print('Writing Stokes drift profile to {:}'.format(run_dir))
    os.makedirs(run_dir, exist_ok=True)
    dat_dump_pfl(timeSD, z, [us_arr, vs_arr], os.path.join(run_dir, 'us_prof.dat'))
    dat_dump_pfl(timeSD, zi, [dusdz_arr, dvsdz_arr], os.path.join(run_dir, 'dusdz_prof.dat'))
    

KPPLT-LF17_L18_Rlx0
KPPLT-LF17_L18_Rlx60
KPPLT-LF17_L18_Rlx600
KPPLT-LF17_L18_Rlx3600
KPPLT-LF17_L36_Rlx0
KPPLT-LF17_L36_Rlx60
KPPLT-LF17_L36_Rlx600
KPPLT-LF17_L36_Rlx3600
KPPLT-LF17_L72_Rlx0
KPPLT-LF17_L72_Rlx60
KPPLT-LF17_L72_Rlx600
KPPLT-LF17_L72_Rlx3600
KPPLT-LF17_L144_Rlx0
KPPLT-LF17_L144_Rlx60
KPPLT-LF17_L144_Rlx600
KPPLT-LF17_L144_Rlx3600
KPPLT-LF17_L288_Rlx0
KPPLT-LF17_L288_Rlx60
KPPLT-LF17_L288_Rlx600
KPPLT-LF17_L288_Rlx3600
KPPLT-LF17-E_L18_Rlx0
KPPLT-LF17-E_L18_Rlx60
KPPLT-LF17-E_L18_Rlx600
KPPLT-LF17-E_L18_Rlx3600
KPPLT-LF17-E_L36_Rlx0
KPPLT-LF17-E_L36_Rlx60
KPPLT-LF17-E_L36_Rlx600
KPPLT-LF17-E_L36_Rlx3600
KPPLT-LF17-E_L72_Rlx0
KPPLT-LF17-E_L72_Rlx60
KPPLT-LF17-E_L72_Rlx600
KPPLT-LF17-E_L72_Rlx3600
KPPLT-LF17-E_L144_Rlx0
KPPLT-LF17-E_L144_Rlx60
KPPLT-LF17-E_L144_Rlx600
KPPLT-LF17-E_L144_Rlx3600
KPPLT-LF17-E_L288_Rlx0
KPPLT-LF17-E_L288_Rlx60
KPPLT-LF17-E_L288_Rlx600
KPPLT-LF17-E_L288_Rlx3600
SMCLT-H15_L18_Rlx0
SMCLT-H15_L18_Rlx60
SMCLT-H15_L18_Rlx600
SMCLT-H15_L18_Rlx3600
SM

## Run the model

In [15]:
%%time
sims = m.run_batch(configs=cfgs, labels=labels, nproc=16)

CPU times: user 279 ms, sys: 91.8 ms, total: 370 ms
Wall time: 1min 3s
